### Map results back to shapefile
The functions below prepare the data for the process of mapping the results back to their original shapefiles by implementing a "table join" between the outputs .csv and the attribute table of the shapefile

In [107]:
import pandas as pd
import os, sys
#sys.path.append(r'C:\Users\gost_\Desktop\lima\GOSTNets\GOSTNets')
sys.path.append(r'../../../GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
importlib.reload(gn)
import geopandas as gpd
import rasterio
from rasterio import features
from shapely.wkt import loads
import numpy as np

networkx version: 2.2 
osmnx version: 0.9 


In [108]:
#define the paths to the origins, destinations and .pickle file
#pth = r'C:\Users\gost_\Desktop\lima\data\OD'
pth = r'../../../../lima_optimization_output'
pckle = r'biggest_subg.pickle'
walk_speed = 4 
WGS = {'init':'epsg:4326'}
measure_crs = {'init':'epsg:32718'}

In [109]:
# read  file
OD_name = r'saved_OD.csv'
OD = pd.read_csv(os.path.join(pth, OD_name), sep=',')
OD = OD.rename(columns = {'Unnamed: 0':'O_ID'})

In [110]:
#preview
OD[:5]

,O_ID,6048,2048,6691,4198,4647,4233,3914,2959,175,3409,474,367,1556,917,4919,4154,6107
0,6147,1968.655016,1020.721567,363.676689,1322.002788,1578.076932,1803.247252,806.455001,2049.990821,1517.705535,1879.396717,1736.154601,803.156991,1542.402922,1657.102815,583.090841,819.749517,1584.512721
1,2052,525.817013,558.842693,1633.650766,274.594031,839.743363,1166.553316,1578.626774,872.524426,1600.311304,1019.720100,666.975509,1744.228690,875.417373,694.618499,1785.150976,1591.881656,544.361984
2,3,1330.771618,448.775456,1230.200991,873.074022,1498.688595,1962.518259,1307.025267,1668.335516,2097.397326,1815.601711,1471.197652,1546.804731,1534.362605,1386.775967,1520.879916,1380.165021,1236.519452
3,6154,153.550210,1138.247790,1658.305553,853.999129,1304.180223,723.313264,1988.980369,429.130522,1197.635321,576.396716,817.834594,1787.841266,1304.052071,1079.897504,1828.763553,1700.636369,822.268208
4,6162,202.318076,907.367997,1799.378687,623.119336,1074.262074,845.305328,1927.152078,551.122585,1319.627384,698.388780,653.254488,1751.140240,1074.133922,849.979356,1792.062527,1663.935343,592.350060


In [111]:
OD_ = OD.set_index('O_ID')
#replacing infinity and negative infinity with not a number
OD = OD_.replace([np.inf, -np.inf], np.nan)
#adding a column that has the minimum time to reach each origin's nearest facility
OD['Minimum'] = OD.loc[:,:].min(axis=1)

In [112]:
OD[:5]

,6048,2048,6691,4198,4647,4233,3914,2959,175,3409,474,367,1556,917,4919,4154,6107,Minimum
O_ID,,,,,,,,,,,,,,,,,,
6147,1968.655016,1020.721567,363.676689,1322.002788,1578.076932,1803.247252,806.455001,2049.990821,1517.705535,1879.396717,1736.154601,803.156991,1542.402922,1657.102815,583.090841,819.749517,1584.512721,363.676689
2052,525.817013,558.842693,1633.650766,274.594031,839.743363,1166.553316,1578.626774,872.524426,1600.311304,1019.720100,666.975509,1744.228690,875.417373,694.618499,1785.150976,1591.881656,544.361984,274.594031
3,1330.771618,448.775456,1230.200991,873.074022,1498.688595,1962.518259,1307.025267,1668.335516,2097.397326,1815.601711,1471.197652,1546.804731,1534.362605,1386.775967,1520.879916,1380.165021,1236.519452,448.775456
6154,153.550210,1138.247790,1658.305553,853.999129,1304.180223,723.313264,1988.980369,429.130522,1197.635321,576.396716,817.834594,1787.841266,1304.052071,1079.897504,1828.763553,1700.636369,822.268208,153.550210
6162,202.318076,907.367997,1799.378687,623.119336,1074.262074,845.305328,1927.152078,551.122585,1319.627384,698.388780,653.254488,1751.140240,1074.133922,849.979356,1792.062527,1663.935343,592.350060,202.318076


In [123]:
### PREPARE ORIGIN FILE 
# set index to nearest node to match on the travel time from that node to nearest destination (OD['Minimum'])
orig = pd.read_csv(os.path.join(pth, 'origins_snapped.csv'))
#orig = orig.rename({'ID':'OBJECTID'}, axis = 1) # rename ID column to 'OBJECTID'
orig = orig.set_index('NN') # used for matching on the OD matrix
orig[:5]

,Unnamed: 0,O_ID,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,NN_dist
NN,,,,,,,,,,,
6528,0,1,3.0,0.003205,2.860800e-07,0,1078.0,-12.24839,-76.91749,POINT (-76.91749 -12.24839),21.347245
5270,1,2,3.0,0.008010,2.565700e-06,1,2374.0,-12.24319,-76.92767,POINT (-76.92766999999999 -12.24319),37.506920
1921,2,3,3.0,0.010808,2.141710e-06,2,367.0,-12.24444,-76.91685,POINT (-76.91685 -12.24444),6.260416
3047,3,4,3.0,0.004083,7.910200e-07,3,1230.0,-12.24269,-76.93012,POINT (-76.93012 -12.24269),34.625022
4378,4,5,3.0,0.006058,1.065810e-06,4,169.0,-12.24249,-76.92808,POINT (-76.92808000000001 -12.24249),27.773978


In [127]:
#orig['NN']

In [128]:
#need to relook at this
#join works on the index as default 
join = orig.join(OD)
join

,Unnamed: 0,O_ID,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,...,175,3409,474,367,1556,917,4919,4154,6107,Minimum
3,633,634,4.0,0.028739,8.435590e-06,633,1458.0,-12.23372,-76.94772,POINT (-76.94771999999999 -12.23372),...,2097.397326,1815.601711,1471.197652,1546.804731,1534.362605,1386.775967,1520.879916,1380.165021,1236.519452,448.775456
21,488,489,3.0,0.005872,1.814450e-06,488,2232.0,-12.19822,-76.96106,POINT (-76.96106 -12.19822),...,1102.641524,680.047647,1031.775449,1902.078238,1525.906486,1304.225116,1943.000525,1814.873341,1073.514728,509.757316
32,89,90,3.0,0.005751,1.013350e-06,89,2041.0,-12.23260,-76.91658,POINT (-76.91658000000001 -12.2326),...,1153.011713,1514.702896,1371.460779,278.204115,1177.709101,1362.446221,295.462617,455.055695,1511.728358,278.204115
82,549,550,3.0,0.003212,6.469200e-07,549,1508.0,-12.19633,-76.94200,POINT (-76.94199999999999 -12.19633),...,422.878004,466.052172,777.169968,1037.260113,715.719204,826.848555,1078.182399,950.055216,973.267778,389.902708
84,212,213,3.0,0.003279,6.724400e-07,212,1610.0,-12.22342,-76.94540,POINT (-76.94540000000001 -12.22342),...,1715.993081,1138.474966,785.389131,1437.392600,880.688869,699.889995,1478.314887,1271.913024,549.633480,183.718794
99,422,423,3.0,0.004647,1.284970e-06,422,1295.0,-12.20459,-76.95295,POINT (-76.95295 -12.20459),...,901.329151,280.090546,553.145249,1515.711259,1047.276286,825.333230,1556.633546,1428.506362,603.017870,90.873706
106,112,113,3.0,0.009651,3.008850e-06,112,1216.0,-12.23132,-76.93962,POINT (-76.93961999999999 -12.23132),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,307,308,3.0,0.013462,4.455160e-06,307,824.0,-12.21750,-76.93547,POINT (-76.93547 -12.2175),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,581,582,3.0,0.008233,2.820590e-06,581,440.0,-12.19100,-76.95468,POINT (-76.95468000000001 -12.191),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,135,136,3.0,0.009341,2.233190e-06,135,1104.0,-12.23721,-76.94012,POINT (-76.94011999999999 -12.23721),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
# add on walktime for time computation
walkspeed = 4
join['walktime'] = join['NN_dist'] / 1000 / walkspeed * 3600
join['total_time'] = (join['walktime'] + join['Minimum']) / 60
###   OR   ###
#compute total distance: NN_dist + dist
join['total_distance'] = (join['NN_dist'] + join['Minimum'])

In [93]:
#read in the original origins (neighborhood polygons) in shapefile format
#fp = r'C:\Users\gost_\Desktop\lima'
fp = r'../../Peru_Lima_Optimization/test_data'
shpfil = gpd.read_file(os.path.join(fp, 'VillaElSalvador_urban.shp'))

In [131]:
#shpfil.index.name
#join.index.name
join[:5]

,Unnamed: 0,O_ID,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,...,367,1556,917,4919,4154,6107,Minimum,walktime,total_time,total_distance
3,633,634,4.0,0.028739,8.435590e-06,633,1458.0,-12.23372,-76.94772,POINT (-76.94771999999999 -12.23372),...,1546.804731,1534.362605,1386.775967,1520.879916,1380.165021,1236.519452,448.775456,17.399312,7.769579,468.108025
21,488,489,3.0,0.005872,1.814450e-06,488,2232.0,-12.19822,-76.96106,POINT (-76.96106 -12.19822),...,1902.078238,1525.906486,1304.225116,1943.000525,1814.873341,1073.514728,509.757316,57.430413,9.453129,573.568887
32,89,90,3.0,0.005751,1.013350e-06,89,2041.0,-12.23260,-76.91658,POINT (-76.91658000000001 -12.2326),...,278.204115,1177.709101,1362.446221,295.462617,455.055695,1511.728358,278.204115,23.531383,5.028925,304.350096
82,549,550,3.0,0.003212,6.469200e-07,549,1508.0,-12.19633,-76.94200,POINT (-76.94199999999999 -12.19633),...,1037.260113,715.719204,826.848555,1078.182399,950.055216,973.267778,389.902708,43.996423,7.231652,438.787622
84,212,213,3.0,0.003279,6.724400e-07,212,1610.0,-12.22342,-76.94540,POINT (-76.94540000000001 -12.22342),...,1437.392600,880.688869,699.889995,1478.314887,1271.913024,549.633480,183.718794,41.301943,3.750346,229.609842


In [139]:
# Ensure both shapefile AND origin file have a non-duplicate index to enable matching of results on to the shapefile
if shpfil.index.name != 'OBJECTID':
    shpfil = shpfil.set_index('OBJECTID')
#join.index.names = ['OBJECTID']
join = join.set_index('O_ID')
join[:5]

,Unnamed: 0,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,NN_dist,...,367,1556,917,4919,4154,6107,Minimum,walktime,total_time,total_distance
O_ID,,,,,,,,,,,,,,,,,,,,,
634,633,4.0,0.028739,8.435590e-06,633,1458.0,-12.23372,-76.94772,POINT (-76.94771999999999 -12.23372),19.332569,...,1546.804731,1534.362605,1386.775967,1520.879916,1380.165021,1236.519452,448.775456,17.399312,7.769579,468.108025
489,488,3.0,0.005872,1.814450e-06,488,2232.0,-12.19822,-76.96106,POINT (-76.96106 -12.19822),63.811570,...,1902.078238,1525.906486,1304.225116,1943.000525,1814.873341,1073.514728,509.757316,57.430413,9.453129,573.568887
90,89,3.0,0.005751,1.013350e-06,89,2041.0,-12.23260,-76.91658,POINT (-76.91658000000001 -12.2326),26.145981,...,278.204115,1177.709101,1362.446221,295.462617,455.055695,1511.728358,278.204115,23.531383,5.028925,304.350096
550,549,3.0,0.003212,6.469200e-07,549,1508.0,-12.19633,-76.94200,POINT (-76.94199999999999 -12.19633),48.884915,...,1037.260113,715.719204,826.848555,1078.182399,950.055216,973.267778,389.902708,43.996423,7.231652,438.787622
213,212,3.0,0.003279,6.724400e-07,212,1610.0,-12.22342,-76.94540,POINT (-76.94540000000001 -12.22342),45.891048,...,1437.392600,880.688869,699.889995,1478.314887,1271.913024,549.633480,183.718794,41.301943,3.750346,229.609842


In [140]:
shpfil[:5]

,NC_CLASS,Centr_Long,Centr_Lat,Shape_Leng,Shape_Area,geometry
OBJECTID,,,,,,
1,3.0,291411.942758,8.645182e+06,0.003205,2.860805e-07,"POLYGON ((-76.9167549641702 -12.2482398349004,..."
2,3.0,290309.820276,8.645806e+06,0.008010,2.565702e-06,POLYGON ((-76.92622229552251 -12.2430487070321...
3,3.0,291520.979384,8.645699e+06,0.010808,2.141712e-06,POLYGON ((-76.91690291343866 -12.2445463561293...
4,3.0,290041.875910,8.645860e+06,0.004083,7.910232e-07,POLYGON ((-76.93050875577444 -12.2420243675305...
5,3.0,290264.651133,8.645883e+06,0.006058,1.065815e-06,POLYGON ((-76.92877667140624 -12.2431240549311...


In [142]:
# Prep output shapefile, match on visualization column, save file
shpfil['total_time'] = join['total_time']
shpfil = shpfil.to_crs({'init':'epsg:4326'})
shpfil.to_file(os.path.join(pth, 'lima_origins_w_time.shp'), driver = 'ESRI Shapefile')

In [144]:
# write the road file as a shapefile
from shapely.wkt import loads
edges = pd.read_csv(os.path.join(pth, 'biggest_subg_edges.csv'))
edges['geometry'] = edges['geometry'].apply(loads)
edges_gdf = gpd.GeoDataFrame(edges, geometry = 'geometry', crs = {'init':'epsg:4326'})
edges_gdf.to_file(os.path.join(pth, 'VES_roads.shp'), driver = 'ESRI Shapefile')